Data loading:

In [1]:
label_set = ['Negative', 'Neutral', 'Positive']
STR_TO_NUM = {k: i for i, k in enumerate(label_set)}
NUM_TO_STR = {i:k for i, k in enumerate(label_set)}

import pandas as pd
df = pd.read_json("bcs_polsent.jsonl", orient="records", lines=True)
df["label"] = df.label.apply(lambda s: STR_TO_NUM[s])
df = df[["sentence", "label", "split"]].rename(columns={"sentence": "text", "label":"labels"})
train = df[df.split=="train"].drop(columns=["split"])
dev = df[df.split=="dev"].drop(columns=["split"])
test = df[df.split=="test"].drop(columns=["split"])

Define functions for training and evaluating

In [2]:
def train_model(train_df, model_name):
    #NUM_EPOCH = 15
    if model_name == "EMBEDDIA/crosloengual-bert":
        model_type = "bert"
    elif model_name == "classla/bcms-bertic":
        model_type = "electra"
        NUM_EPOCH = 9
    elif model_name == "xlm-roberta-base":
        model_type = "xlmroberta"
    else:
        raise AttributeError(f"Expected either xlm-roberta-base, classla/bcms-bertic, or EMBEDDIA/crosloengual-bert, got {model_name}.")

    from simpletransformers.classification import ClassificationModel
    import torch
    torch.cuda.empty_cache()
    model_args = {
        "num_train_epochs": NUM_EPOCH,
        "learning_rate": 4e-5,
        "overwrite_output_dir": True,
        "train_batch_size": 8, 
        "no_save": True,
        "no_cache": True,
        "overwrite_output_dir": True,
        "save_steps": -1,
        "max_seq_length": 512,
        "silent": True,
    }

    model = ClassificationModel(
        model_type, model_name, num_labels=3, use_cuda=True, args=model_args
    )
    model.train_model(train_df)
    return model


def eval_model(model, test_df):
    y_true_enc = test_df.labels
    from tqdm.auto import tqdm
    y_pred_enc = [model.predict(i)[0][0] for i in tqdm(test_df.text.values)]

    y_true = [NUM_TO_STR[i] for i in y_true_enc]
    y_pred = [NUM_TO_STR[i] for i in y_pred_enc]
    from sklearn.metrics import f1_score
    macroF1 = f1_score(y_true, y_pred, labels=label_set, average="macro")

    return {"macroF1": macroF1, "y_true": y_true, "y_pred": y_pred}


In [3]:
#models = "xlm-roberta-base classla/bcms-bertic EMBEDDIA/crosloengual-bert".split()
models = ["classla/bcms-bertic"]
from tqdm.auto import tqdm
for model_name in tqdm(models*6):
    # Full train-test:
    model = train_model(train, model_name)
    df = pd.read_json("bcs_polsent.jsonl", orient="records", lines=True)
    df["label"] = df.label.apply(lambda s: STR_TO_NUM[s])
    df = df[["sentence", "label", "split", "country"]].rename(columns={"sentence": "text", "label":"labels"})
    train = df[df.split=="train"].drop(columns=["split", "country"])
    condition_is_test = df.split=="test"
    test = df[condition_is_test].drop(columns=["split", "country"])
    stats = eval_model(model, test)
    stats["train_split"] = "train"
    stats["eval_split"] = "test"
    stats["model_name"] = model_name
    with open("005_results.jsonl", "a") as f:
        f.write(f"{stats}\n")
        
    # Eval on specific countries
    for country in ["HR", "SRB"]:
        import pandas as pd
        df = pd.read_json("bcs_polsent.jsonl", orient="records", lines=True)
        df["label"] = df.label.apply(lambda s: STR_TO_NUM[s])
        df = df[["sentence", "label", "split", "country"]].rename(columns={"sentence": "text", "label":"labels"})
        condition_is_test = df.split=="test"
        condition_is_lang = df.country == country
        test = df[condition_is_test & condition_is_lang].drop(columns=["split", "country"])
        stats = eval_model(model, test)
        stats["train_split"] = "train"
        stats["eval_split"] = f"test_{country}"
        stats["model_name"] = model_name
        with open("005_results.jsonl", "a") as f:
            f.write(f"{stats}\n")

  0%|          | 0/6 [00:00<?, ?it/s]

Some weights of the model checkpoint at classla/bcms-bertic were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at classla/bcms-bertic and are newly initialized: ['classifier.dense.weight', 'classifier.ou

  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/163 [00:00<?, ?it/s]

  0%|          | 0/119 [00:00<?, ?it/s]

...

  0%|          | 0/119 [00:00<?, ?it/s]

Some weights of the model checkpoint at classla/bcms-bertic were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at classla/bcms-bertic and are newly initialized: ['classifier.dense.weight', 'classifier.ou

  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/163 [00:00<?, ?it/s]

  0%|          | 0/119 [00:00<?, ?it/s]

In [4]:
# models = "xlm-roberta-base classla/bcms-bertic EMBEDDIA/crosloengual-bert".split()
# from tqdm.auto import tqdm
# for model_name in tqdm(models*5):
#     model = train_model(train, model_name)
#     stats = eval_model(model, test)
#     stats["train_split"] = "train"
#     stats["eval_split"] = "test"
#     stats["model_name"] = model_name
#     with open("005_results.jsonl", "a") as f:
#         f.write(f"{stats}\n")

# Train on train, evaluate on specific country

This was done already in the cell above.

In [5]:
# models = "xlm-roberta-base classla/bcms-bertic EMBEDDIA/crosloengual-bert".split()
# from tqdm.auto import tqdm
# for model_name in tqdm(models*6):
#     model = train_model(train, model_name)
#     for country in ["HR", "SRB"]:
#         import pandas as pd
#         df = pd.read_json("bcs_polsent.jsonl", orient="records", lines=True)
#         df["label"] = df.label.apply(lambda s: STR_TO_NUM[s])
#         df = df[["sentence", "label", "split", "country"]].rename(columns={"sentence": "text", "label":"labels"})
#         condition_is_test = df.split=="test"
#         condition_is_lang = df.country == country
#         test = df[condition_is_test & condition_is_lang].drop(columns=["split", "country"])
#         stats = eval_model(model, test)
#         stats["train_split"] = "train"
#         stats["eval_split"] = f"test_{country}"
#         stats["model_name"] = model_name
#         with open("005_results.jsonl", "a") as f:
#             f.write(f"{stats}\n")

# Train on specific country, evaluate on specific country:

The implementation is less than elegant (by light years), but this was written when the training was already underway and reformatting would require restarting.

In [6]:
#models = "xlm-roberta-base classla/bcms-bertic EMBEDDIA/crosloengual-bert".split()
models = ["classla/bcms-bertic"]
from tqdm.auto import tqdm
for model_name in tqdm(models*6):
    for country_train in ["HR", "SRB"]:
        df = pd.read_json("bcs_polsent.jsonl", orient="records", lines=True)
        df["label"] = df.label.apply(lambda s: STR_TO_NUM[s])
        df = df[["sentence", "label", "split", "country"]].rename(columns={"sentence": "text", "label":"labels"})
        condition_is_train = df.split=="train"
        condition_is_lang = df.country == country_train
        train = df[condition_is_train & condition_is_lang].drop(columns=["split", "country"])
        model = train_model(train, model_name)
        for country_eval in ["HR", "SRB"]:
            df = pd.read_json("bcs_polsent.jsonl", orient="records", lines=True)
            df["label"] = df.label.apply(lambda s: STR_TO_NUM[s])
            df = df[["sentence", "label", "split", "country"]].rename(columns={"sentence": "text", "label":"labels"})
            condition_is_test = df.split=="test"
            condition_is_lang = df.country == country_eval
            test = df[condition_is_test & condition_is_lang].drop(columns=["split", "country"])
            stats = eval_model(model, test)
            stats["train_split"] = f"train_{country_train}"
            stats["eval_split"] = f"test_{country_eval}"
            stats["model_name"] = model_name
            with open("005_results.jsonl", "a") as f:
                f.write(f"{stats}\n")

  0%|          | 0/6 [00:00<?, ?it/s]

Some weights of the model checkpoint at classla/bcms-bertic were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at classla/bcms-bertic and are newly initialized: ['classifier.dense.weight', 'classifier.ou

  0%|          | 0/163 [00:00<?, ?it/s]

  0%|          | 0/119 [00:00<?, ?it/s]

Some weights of the model checkpoint at classla/bcms-bertic were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at classla/bcms-bertic and are newly initialized: ['classifier.dense.weight', 'classifier.ou

...

  0%|          | 0/119 [00:00<?, ?it/s]

Some weights of the model checkpoint at classla/bcms-bertic were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at classla/bcms-bertic and are newly initialized: ['classifier.dense.weight', 'classifier.ou

  0%|          | 0/163 [00:00<?, ?it/s]

  0%|          | 0/119 [00:00<?, ?it/s]

Some weights of the model checkpoint at classla/bcms-bertic were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at classla/bcms-bertic and are newly initialized: ['classifier.dense.weight', 'classifier.ou